<a href="https://colab.research.google.com/github/joyfulspace/ADP/blob/master/06_%ED%86%B5%EA%B3%84%EB%B6%84%EC%84%9D_4_%ED%9A%8C%EA%B7%80%EB%B3%80%EC%88%98%EC%84%A0%ED%83%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/joyfulspace/ADP.git

Cloning into 'ADP'...
remote: Enumerating objects: 2593, done.
remote: Counting objects: 100% (456/456), done.
remote: Compressing objects: 100% (264/264), done.
remote: Total 2593 (delta 198), reused 426 (delta 183), pack-reused 2137
Receiving objects: 100% (2593/2593), 58.96 MiB | 7.43 MiB/s, done.
Resolving deltas: 100% (202/202), done.
Checking out files: 100% (2407/2407), done.


# 회귀변수선택
- 전진선택법(Forward Selction)
- 후진소거전(Backward Elimination)
- 단계적선택법(Stepwise Selection)
  
[출처] https://todayisbetterthanyesterday.tistory.com/10

In [2]:
import time
import itertools

import pandas as pd
import numpy as np

import statsmodels.api as sm
from sklearn.model_selection import train_test_split

## 데이터 불러오기 및 전처리 

In [3]:
data_df = pd.read_csv('ADP/data/ToyotaCorolla.csv')

In [4]:
data_df.shape

(1436, 37)

In [5]:
data_df.head()

,Id,Model,Price,Age_08_04,Mfg_Month,Mfg_Year,KM,Fuel_Type,HP,Met_Color,...,Central_Lock,Powered_Windows,Power_Steering,Radio,Mistlamps,Sport_Model,Backseat_Divider,Metallic_Rim,Radio_cassette,Tow_Bar
0,1,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13500,23,10,2002,46986,Diesel,90,1,...,1,1,1,0,0,0,1,0,0,0
1,2,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13750,23,10,2002,72937,Diesel,90,1,...,1,0,1,0,0,0,1,0,0,0
2,3,?TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13950,24,9,2002,41711,Diesel,90,1,...,0,0,1,0,0,0,1,0,0,0
3,4,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,14950,26,7,2002,48000,Diesel,90,0,...,0,0,1,0,0,0,1,0,0,0
4,5,TOYOTA Corolla 2.0 D4D HATCHB SOL 2/3-Doors,13750,30,3,2002,38500,Diesel,90,0,...,1,1,1,0,1,0,1,0,0,0


In [6]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1436 entries, 0 to 1435
Data columns (total 37 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Id                1436 non-null   int64 
 1   Model             1436 non-null   object
 2   Price             1436 non-null   int64 
 3   Age_08_04         1436 non-null   int64 
 4   Mfg_Month         1436 non-null   int64 
 5   Mfg_Year          1436 non-null   int64 
 6   KM                1436 non-null   int64 
 7   Fuel_Type         1436 non-null   object
 8   HP                1436 non-null   int64 
 9   Met_Color         1436 non-null   int64 
 10  Automatic         1436 non-null   int64 
 11  cc                1436 non-null   int64 
 12  Doors             1436 non-null   int64 
 13  Cylinders         1436 non-null   int64 
 14  Gears             1436 non-null   int64 
 15  Quarterly_Tax     1436 non-null   int64 
 16  Weight            1436 non-null   int64 
 17  Mfr_Guarantee 

In [7]:
data_df.Fuel_Type.unique()

array(['Diesel', 'Petrol', 'CNG'], dtype=object)

In [8]:
# Model과 Fuel_Type만 object. Model은 Id로 대체할 수 있고 Fuel_Type은 정수형으로 변경하기
mlr_data = pd.concat([data_df, pd.get_dummies(data_df.Fuel_Type)], axis=1)

# 불필요한 변수 제거
mlr_data = mlr_data.drop(['Id', 'Model', 'Fuel_Type'], axis=1)

In [9]:
mlr_data.head()

,Price,Age_08_04,Mfg_Month,Mfg_Year,KM,HP,Met_Color,Automatic,cc,Doors,...,Radio,Mistlamps,Sport_Model,Backseat_Divider,Metallic_Rim,Radio_cassette,Tow_Bar,CNG,Diesel,Petrol
0,13500,23,10,2002,46986,90,1,0,2000,3,...,0,0,0,1,0,0,0,0,1,0
1,13750,23,10,2002,72937,90,1,0,2000,3,...,0,0,0,1,0,0,0,0,1,0
2,13950,24,9,2002,41711,90,1,0,2000,3,...,0,0,0,1,0,0,0,0,1,0
3,14950,26,7,2002,48000,90,0,0,2000,3,...,0,0,0,1,0,0,0,0,1,0
4,13750,30,3,2002,38500,90,0,0,2000,3,...,0,1,0,1,0,0,0,0,1,0


In [10]:
# 상수항 추가(y절편)
mlr_data = sm.add_constant(mlr_data, has_constant='add')

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


## 데이터 분할
- X, Y 데이터 설정 및 학습, 평가 데이터 분리

In [11]:
mlr_data.columns.difference(['Price'])

Index(['ABS', 'Age_08_04', 'Airbag_1', 'Airbag_2', 'Airco', 'Automatic',
       'Automatic_airco', 'BOVAG_Guarantee', 'Backseat_Divider',
       'Boardcomputer', 'CD_Player', 'CNG', 'Central_Lock', 'Cylinders',
       'Diesel', 'Doors', 'Gears', 'Guarantee_Period', 'HP', 'KM', 'Met_Color',
       'Metallic_Rim', 'Mfg_Month', 'Mfg_Year', 'Mfr_Guarantee', 'Mistlamps',
       'Petrol', 'Power_Steering', 'Powered_Windows', 'Quarterly_Tax', 'Radio',
       'Radio_cassette', 'Sport_Model', 'Tow_Bar', 'Weight', 'cc', 'const'],
      dtype='object')

In [12]:
feature_columns = mlr_data.columns.difference(['Price']) # Target column 빼기

X = mlr_data[feature_columns]
y = mlr_data.Price

train_x, test_x, train_y, test_y = train_test_split(X, y, train_size=0.7, test_size=0.3)
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

(1005, 37) (431, 37) (1005,) (431,)


## 회귀 모델 학습

In [13]:
full_model = sm.OLS(train_y, train_x)
fitted_full_model = full_model.fit()
fitted_full_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.916
Model:                            OLS   Adj. R-squared:                  0.914
Method:                 Least Squares   F-statistic:                     322.4
Date:                Tue, 06 Sep 2022   Prob (F-statistic):               0.00
Time:                        12:22:36   Log-Likelihood:                -8418.0
No. Observations:                1005   AIC:                         1.690e+04
Df Residuals:                     971   BIC:                         1.707e+04
Df Model:                          33                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
ABS               -148.3082    131.770     -1.126      0.261    -406.895     110.278
Age_08_04         -115.1334      4.076    -28.245      0.000    -123.133    -107.134
Airbag_1            22.0065    249.853      0.088      0.930    -468.308     512.321
Airbag_2          -194.9953    131.810     -1.479      0.139    -453.661      63.670
Airco              236.6524     91.429      2.588      0.010      57.230     416.074
Automatic          150.6911    170.366      0.885      0.377    -183.637     485.019
Automatic_airco   2330.0109    196.675     11.847      0.000    1944.053    2715.969
BOVAG_Guarantee    557.1177    127.510      4.369      0.000     306.890     807.346
Backseat_Divider  -242.9541    130.729     -1.858      0.063    -499.497      13.589
Boardcomputer     -373.2482    123.793     -3.015      0.003    -616.182    -130.315
CD_Player          257.8939    102.240      2.522      0.012      57.258     458.530
CNG              -1104.6175    215.315     -5.130      0.000   -1527.155    -682.080
Central_Lock      -351.3334    146.371     -2.400      0.017    -638.574     -64.093
Cylinders           -0.0368      0.002    -15.708      0.000      -0.041      -0.032
Diesel            -374.4225    191.854     -1.952      0.051    -750.918       2.073
Doors               -7.3431     41.040     -0.179      0.858     -87.881      73.195
Gears               57.4182    220.200      0.261      0.794    -374.705     489.542
Guarantee_Period    57.3754     13.503      4.249      0.000      30.876      83.874
HP                  12.7276      3.745      3.399      0.001       5.379      20.077
KM                  -0.0165      0.001    -12.271      0.000      -0.019      -0.014
Met_Color          -40.7379     78.264     -0.521      0.603    -194.324     112.849
Metallic_Rim       177.8069     96.795      1.837      0.067     -12.144     367.758
Mfg_Month          -81.9573     10.362     -7.909      0.000    -102.292     -61.622
Mfg_Year            -2.0062      0.914     -2.194      0.028      -3.801      -0.212
Mfr_Guarantee      253.7317     75.534      3.359      0.001     105.503     401.960
Mistlamps         -122.6333    111.493     -1.100      0.272    -341.428      96.162
Petrol            1479.0308    198.684      7.444      0.000    1089.131    1868.930
Power_Steering     198.2188    286.002      0.693      0.488    -363.034     759.471
Powered_Windows    659.1448    146.936      4.486      0.000     370.795     947.494
Quarterly_Tax       14.3575      1.883      7.623      0.000      10.662      18.053
Radio              601.8646    635.411      0.947      0.344    -645.073    1848.803
Radio_cassette    -662.2803    635.438     -1.042      0.298   -1909.271     584.710
Sport_Model        217.9285     90.670      2.404      0.016      39.996     395.861
Tow_Bar           -216.

In [14]:
# 변수가 많아 하나하나에 대한 신뢰도가 떨어짐
# VIF를 통한 다중공선성 확인
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif['VIF Factor'] = [variance_inflation_factor(mlr_data.values, i) for i in range(mlr_data.shape[1])]
vif['features'] = mlr_data.columns

/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.7/dist-packages/statsmodels/stats/outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


In [15]:
vif.head()

,VIF Factor,features
0,0.000000,const
1,10.953474,Price
2,inf,Age_08_04
3,inf,Mfg_Month
4,inf,Mfg_Year


p-value가 유의하지 않고 vif가 높으면 중요하지 않은 변수로 선택해 제거. 다만 그래도 논리적으로 중요한 변수라면 지우지 않고 살린다. 여기서는 변수선택과정을 통해 선택

# 0. 변수선택과정 함수화.

In [16]:
# 선택된 변수(feature_set)으로 모델 형성하고 AIC 구하는 함수
def processSubset(X, y, feature_set):
  model = sm.OLS(y, X[list(feature_set)]) # modeling
  regr = model.fit()                      # model fitting
  AIC = regr.aic                          # model's AIC
  return {'model':regr, 'AIC':AIC}

# 가장 낮은 AIC를 가지는 모델 선택, 저장
def getBest(X, y, k):
  tic = time.time()
  results = []

  # 상수항 제외하고 k개씩 조합해 AIC 계산하기
  for combo in itertools.combinations(train_x.columns.difference(['const']), k):
    combo = (list(combo)+['const'])
    results.append(processSubset(X, y, feature_set=combo))  # 모델링결과 저장
  models = pd.DataFrame(results)
  
  # AIC가 가장 작은 모델 선택
  best_model = models.loc[models['AIC'].argmin()]

  toc = time.time()
  print('Processed', models.shape[0], 'models on', k, 'predictors in', (toc-tic), 'seconds.')

  return best_model

In [17]:
# 전체 변수의 AIC
print(processSubset(X=train_x, y=train_y, feature_set=feature_columns))

{'model': <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x7ff7b08c81d0>, 'AIC': 16904.01260980929}


In [18]:
# 두 개 변수 선택 시, 최고 모델
getBest(X=train_x, y=train_y, k=2)

Processed 630 models on 2 predictors in 1.0275194644927979 seconds.


model    <statsmodels.regression.linear_model.Regressio...
AIC                                           17544.065878
Name: 211, dtype: object

In [19]:
# 변수 개수에 따라 비교
models = pd.DataFrame(columns=['AIC', 'model'])
for i in range(1, 4):
  models.loc[i] = getBest(X=train_x, y=train_y, k=i)

Processed 36 models on 1 predictors in 0.060231685638427734 seconds.
Processed 630 models on 2 predictors in 0.868410587310791 seconds.
Processed 7140 models on 3 predictors in 11.062219142913818 seconds.


In [20]:
models

,AIC,model
1,17820.329918,<statsmodels.regression.linear_model.Regressio...
2,17544.065878,<statsmodels.regression.linear_model.Regressio...
3,17309.041958,<statsmodels.regression.linear_model.Regressio...


In [21]:
# 모든 변수를 사용한 것과 비교
print("full model Rsquared:","{:.5f}".format(fitted_full_model.rsquared))
print("full model AIC:","{:.5f}".format(fitted_full_model.aic))
print("full model MSE:","{:.5f}".format(fitted_full_model.mse_total))

print("selected model Rsquared:","{:.5f}".format(models.loc[3,"model"].rsquared))
print("selected model AIC:","{:.5f}".format(models.loc[3,"model"].aic))
print("selected model MSE:","{:.5f}".format(models.loc[3,"model"].mse_total))

full model Rsquared: 0.91636
full model AIC: 16904.01261
full model MSE: 13211459.61808
selected model Rsquared: 0.86715
selected model AIC: 17309.04196
selected model MSE: 13211459.61808


# 1. 전진선택법

In [37]:
# 전진선택법
def forward(X, y, predictors):
  # predictor: 선택한 변수
  # 이미 선택된 변수(predictor)와 상수(const)를 제외한 변수
  remaining_predictors = [p for p in X.columns.difference(['const']) \
                          if p not in predictors]
  tic = time.time()
  results = []
  # 이미 선택된 변수와 상수에 1개 변수를 추가한 결과를 results에 저장
  for p in remaining_predictors:
    results.append(processSubset(X=X, y=y, feature_set=predictors+[p]+['const']))                        

  models = pd.DataFrame(results)

  # AIC가 가장 낮은 모델 선택
  best_model = models.loc[models['AIC'].argmin()]
  toc = time.time()
  print('Processed', models.shape[0], 'models on', 
        len(predictors)+1, 'predictors in', (toc-tic))
  print('Selected predictors:', best_model['model'].model.exog_names, 
        'AIC:', best_model[0].aic)
  return best_model

def forward_model(X, y):
  Fmodels = pd.DataFrame(columns=['AIC', 'model'])
  tic = time.time()

  predictors = []
  for i in range(1, len(X.columns.difference(['const']))+1):
    # 현재 변수들로 계산된 가장 작은 AIC
    Forward_result = forward(X=X, y=y, predictors=predictors)
    if i>1:
      # AIC가 이전단계보다 크면 중단
      if Forward_result['AIC'] > Fmodel_before:
        break
    Fmodels.loc[i] = Forward_result
    predictors = Fmodels.loc[i, 'model'].model.exog_names # 선택된 모델의 변수 (AIC가 가장 작은 모델의 변수)
    Fmodel_before = Fmodels.loc[i, 'AIC']
    predictors = [k for k in predictors if k!='const'] # 상수 제외
  toc = time.time()
  print('Total elapsed time:', toc-tic, 'seconds')

  return Fmodels['model'][len(Fmodels['model'])]

In [23]:
Forward_best_model = forward_model(X=train_x, y=train_y)

Processed 36 models on 1 predictors in 0.06354308128356934
Selected predictors: ['Mfg_Year', 'const'] AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x7ff7b040ec10>
Processed 35 models on 2 predictors in 0.05497169494628906
Selected predictors: ['Mfg_Year', 'Automatic_airco', 'const'] AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x7ff7b040e110>
Processed 34 models on 3 predictors in 0.05870342254638672
Selected predictors: ['Mfg_Year', 'Automatic_airco', 'Weight', 'const'] AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x7ff7b06cba90>
Processed 33 models on 4 predictors in 0.05435800552368164
Selected predictors: ['Mfg_Year', 'Automatic_airco', 'Weight', 'KM', 'const'] AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x7ff7b065b1d0>
Processed 32 models on 5 predictors in 0.06784534454345703
Selected predictors: ['Mfg_Year', 'Automatic_airco', 'Weight', 'KM', 'HP', 'co

In [24]:
Forward_best_model.aic

16889.20344363156

In [25]:
Forward_best_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.916
Model:                            OLS   Adj. R-squared:                  0.914
Method:                 Least Squares   F-statistic:                     485.2
Date:                Tue, 06 Sep 2022   Prob (F-statistic):               0.00
Time:                        12:22:52   Log-Likelihood:                -8421.6
No. Observations:                1005   AIC:                         1.689e+04
Df Residuals:                     982   BIC:                         1.700e+04
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Mfg_Year           983.3023    123.488      7.963      0.000     740.971    1225.634
Automatic_airco   2272.3602    185.307     12.263      0.000    1908.717    2636.004
Weight              16.8116      1.248     13.468      0.000      14.362      19.261
KM                  -0.0167      0.001    -12.614      0.000      -0.019      -0.014
HP                  12.1024      3.537      3.422      0.001       5.161      19.043
Powered_Windows    647.5524    144.753      4.473      0.000     363.491     931.614
BOVAG_Guarantee    568.1064    125.376      4.531      0.000     322.071     814.141
Guarantee_Period    56.6314     13.385      4.231      0.000      30.365      82.898
Mfr_Guarantee      246.4825     74.161      3.324      0.001     100.950     392.015
Petrol            1900.7018    301.605      6.302      0.000    1308.836    2492.567
Quarterly_Tax       14.3136      1.833      7.808      0.000      10.716      17.911
Boardcomputer     -349.0024    121.631     -2.869      0.004    -587.689    -110.316
Age_08_04          -32.1766     10.396     -3.095      0.002     -52.577     -11.776
Tow_Bar           -216.5120     80.273     -2.697      0.007    -374.038     -58.986
Airbag_2          -289.0047    101.629     -2.844      0.005    -488.439     -89.571
CD_Player          252.7351     98.221      2.573      0.010      59.988     445.482
Central_Lock      -374.7953    143.915     -2.604      0.009    -657.212     -92.379
Airco              214.6059     88.015      2.438      0.015      41.887     387.325
Sport_Model        242.2036     84.835      2.855      0.004      75.726     408.682
CNG               -639.9143    340.784     -1.878      0.061   -1308.663      28.835
Backseat_Divider  -226.2373    124.659     -1.815      0.070    -470.866      18.392
Metallic_Rim       150.4082     90.211      1.667      0.096     -26.620     327.437
const            -1.976e+06   2.47e+05     -7.986      0.000   -2.46e+06   -1.49e+06
==============================================================================
Omnibus:                       80.106   Durbin-Watson:                   1.944
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              374.644
Skew:                           0.170   Prob(JB):                     4.44e-82
Kurtosis:                       5.972   Cond. No.                     5.76e+08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.76e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# 2. 후진소거법

In [35]:
def backward(X, y, predictors):
  tic = time.time()
  results = []

  # 선택된 변수보다 1개 적은 개수의 변수로 조합 만들기
  for combo in itertools.combinations(predictors, len(predictors)-1):
    results.append(processSubset(X=X, y=y, feature_set=list(combo)+['const']))
  models = pd.DataFrame(results)

  # 가장 낮은 AIC를 가진 모델을 선택
  best_model = models.loc[models['AIC'].argmin()]
  toc = time.time()

  print('Processed ', models.shape[0], 'models on', len(predictors)-1, 'predictors in', toc-tic)
  print('Selected predictors:', best_model['model'].model.exog_names, 'AIC:', best_model[0].aic)
  return best_model

def backward_model(X, y):
  Bmodels = pd.DataFrame(columns=['AIC', 'model'], index=range(1, len(X.columns)))

  tic = time.time()
  # 상수를 제외한 모든 변수를 선택한채로 시작
  predictors = X.columns.difference(['const'])
  Bmodel_before = processSubset(X, y, predictors)['AIC']
  while (len(predictors)>1):
    # 현재 변수에서 가장 AIC가 작은 모델 구하기
    Backward_result = backward(X=train_x, y=train_y, predictors=predictors)
    # 이전 모델보다 AIC가 크면 중단
    if Backward_result['AIC'] > Bmodel_before:
      break

    Bmodels.loc[len(predictors)-1] = Backward_result
    predictors = Bmodels.loc[len(predictors)-1]['model'].model.exog_names
    Bmodel_before = Backward_result['AIC']
    predictors = [k for k in predictors if k!='const']

  toc = time.time()
  print('Total elapsed time:', toc-tic, 'seconds.')
  return Bmodels['model'].dropna().iloc[0]

In [27]:
Backward_best_model = backward_model(X=train_x, y=train_y)

Processed  36 models on 35 predictors in 0.4957461357116699
Selected predictors: ['ABS', 'Age_08_04', 'Airbag_2', 'Airco', 'Automatic', 'Automatic_airco', 'BOVAG_Guarantee', 'Backseat_Divider', 'Boardcomputer', 'CD_Player', 'CNG', 'Central_Lock', 'Cylinders', 'Diesel', 'Doors', 'Gears', 'Guarantee_Period', 'HP', 'KM', 'Met_Color', 'Metallic_Rim', 'Mfg_Month', 'Mfg_Year', 'Mfr_Guarantee', 'Mistlamps', 'Petrol', 'Power_Steering', 'Powered_Windows', 'Quarterly_Tax', 'Radio', 'Radio_cassette', 'Sport_Model', 'Tow_Bar', 'Weight', 'cc', 'const'] AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x7ff7b05a6950>
Processed  35 models on 34 predictors in 0.5323851108551025
Selected predictors: ['ABS', 'Age_08_04', 'Airbag_2', 'Airco', 'Automatic', 'Automatic_airco', 'BOVAG_Guarantee', 'Backseat_Divider', 'Boardcomputer', 'CD_Player', 'CNG', 'Central_Lock', 'Cylinders', 'Diesel', 'Gears', 'Guarantee_Period', 'HP', 'KM', 'Met_Color', 'Metallic_Rim', 'Mfg_Month', 'Mfg_Yea

In [28]:
Backward_best_model.aic

16889.20344363156

In [29]:
Backward_best_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.916
Model:                            OLS   Adj. R-squared:                  0.914
Method:                 Least Squares   F-statistic:                     485.2
Date:                Tue, 06 Sep 2022   Prob (F-statistic):               0.00
Time:                        12:22:58   Log-Likelihood:                -8421.6
No. Observations:                1005   AIC:                         1.689e+04
Df Residuals:                     982   BIC:                         1.700e+04
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Age_08_04          -32.1766     10.396     -3.095      0.002     -52.577     -11.776
Airbag_2          -289.0047    101.629     -2.844      0.005    -488.439     -89.571
Airco              214.6059     88.015      2.438      0.015      41.887     387.325
Automatic_airco   2272.3602    185.307     12.263      0.000    1908.717    2636.004
BOVAG_Guarantee    568.1064    125.376      4.531      0.000     322.071     814.141
Backseat_Divider  -226.2373    124.659     -1.815      0.070    -470.866      18.392
Boardcomputer     -349.0024    121.631     -2.869      0.004    -587.689    -110.316
CD_Player          252.7351     98.221      2.573      0.010      59.988     445.482
CNG              -2540.6161    356.931     -7.118      0.000   -3241.052   -1840.180
Central_Lock      -374.7953    143.915     -2.604      0.009    -657.212     -92.379
Cylinders        -4.644e+05   5.82e+04     -7.978      0.000   -5.79e+05    -3.5e+05
Diesel           -1900.7018    301.605     -6.302      0.000   -2492.567   -1308.836
Guarantee_Period    56.6314     13.385      4.231      0.000      30.365      82.898
HP                  12.1024      3.537      3.422      0.001       5.161      19.043
KM                  -0.0167      0.001    -12.614      0.000      -0.019      -0.014
Metallic_Rim       150.4082     90.211      1.667      0.096     -26.620     327.437
Mfg_Year           983.3023    123.488      7.963      0.000     740.971    1225.634
Mfr_Guarantee      246.4825     74.161      3.324      0.001     100.950     392.015
Powered_Windows    647.5524    144.753      4.473      0.000     363.491     931.614
Quarterly_Tax       14.3136      1.833      7.808      0.000      10.716      17.911
Sport_Model        242.2036     84.835      2.855      0.004      75.726     408.682
Tow_Bar           -216.5120     80.273     -2.697      0.007    -374.038     -58.986
Weight              16.8116      1.248     13.468      0.000      14.362      19.261
const            -1.161e+05   1.46e+04     -7.978      0.000   -1.45e+05   -8.75e+04
==============================================================================
Omnibus:                       80.106   Durbin-Watson:                   1.944
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              374.644
Skew:                           0.170   Prob(JB):                     4.44e-82
Kurtosis:                       5.972   Cond. No.                     1.72e+20
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.08e-28. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

# 3. 단계적선택법

In [33]:
def Stepwise_model(X, y):
  Stepmodels = pd.DataFrame(columns=['AIC', 'model'])
  tic = time.time()
  predictors = []
  Smodel_before = processSubset(X, y, predictors+['const'])['AIC']

  for i in range(1, len(X.columns.difference(['const']))+1):
    # 전진선택법(Forward) 실행
    Forward_result = forward(X=X, y=y, predictors=predictors)
    Stepmodels.loc[i] = Forward_result
    Smodel_before = Stepmodels.loc[i, 'AIC']
    predictors = Forward_result.model.model.exog_names
    predictors = [k for k in predictors if k!='const']
    print('forward')

    # 후진소거법(Backward) 실행
    Backward_result = backward(X=X, y=y, predictors=predictors)
    # Backward보다 Forward의 AIC가 더 크다면
    if Backward_result['AIC'] < Forward_result['AIC']:
      Stepmodels.loc[i] = Backward_result
      predictors = Stepmodels.loc[i, 'model'].model.exog_names
      Smodel_before = Stepmodels.loc[i, 'AIC']
      predictors = [k for k in predictors if k!='const']
      print('backward')
    
    if Stepmodels.loc[i]['AIC'] > Smodel_before:
      break
    else:
      Smodel_before = Stepmodels.loc[i]['AIC']

  toc = time.time()
  print('Total elapsed time:', toc-tic, 'seconds.')
  return Stepmodels['model'][len(Stepmodels['model'])]

In [38]:
Stepwise_best_model = Stepwise_model(X=train_x, y=train_y)

Processed 36 models on 1 predictors in 0.053087711334228516
Selected predictors: ['Mfg_Year', 'const'] AIC: 17820.329918474872
forward
Processed  1 models on 0 predictors in 0.0022897720336914062
Selected predictors: ['const'] AIC: 19331.644093532374
Processed 35 models on 2 predictors in 0.05247020721435547
Selected predictors: ['Mfg_Year', 'Automatic_airco', 'const'] AIC: 17544.065877730816
forward
Processed  2 models on 1 predictors in 0.00397491455078125
Selected predictors: ['Mfg_Year', 'const'] AIC: 17820.329918474872
Processed 34 models on 3 predictors in 0.0491032600402832
Selected predictors: ['Mfg_Year', 'Automatic_airco', 'Weight', 'const'] AIC: 17426.071103820977
forward
Processed  3 models on 2 predictors in 0.014143705368041992
Selected predictors: ['Mfg_Year', 'Automatic_airco', 'const'] AIC: 17544.065877730816
Processed 33 models on 4 predictors in 0.062398433685302734
Selected predictors: ['Mfg_Year', 'Automatic_airco', 'Weight', 'KM', 'const'] AIC: 17138.131061827244


In [40]:
Stepwise_best_model.aic

16889.20344363156

# 4. 성능평가

In [43]:
# number of params
print(Forward_best_model.params.shape, Backward_best_model.params.shape, Stepwise_best_model.params.shape)

(23,) (24,) (23,)


In [44]:
# 값 예측. test_y
pred_y_full = fitted_full_model.predict(test_x)
pred_y_forward = Forward_best_model.predict(test_x[Forward_best_model.model.exog_names])
pred_y_backward = Backward_best_model.predict(test_x[Backward_best_model.model.exog_names])
pred_y_stepwise = Stepwise_best_model.predict(test_x[Stepwise_best_model.model.exog_names])

In [46]:
# MSE, RMSE, MAE, MAPE로 성능평가
perf_mat = pd.DataFrame(columns=['ALL', 'FORWARD', 'BACKWARD', 'STEPWISE'],
                        index=['MSE', 'RMSE', 'MAE', 'MAPE'])

# MAPE의 함수
def mean_absolute_percentage_error(y_true, y_pred):
  y_true, y_pred = np.array(y_true), np.array(y_pred)
  return np.mean(np.abs((y_true-y_pred)/y_true))*100

from sklearn import metrics

for col in ['ALL', 'FORWARD', 'BACKWARD', 'STEPWISE']:
    if col=='ALL': temp = pred_y_full.copy()
    elif col=='FORWARD': temp = pred_y_forward.copy()
    elif col=='BACKWARD': temp = pred_y_backward.copy()
    elif col=='STEPWISE': temp = pred_y_stepwise.copy()

    perf_mat.loc['MSE', col] = metrics.mean_squared_error(test_y, temp)
    perf_mat.loc['RMSE', col] = np.sqrt(metrics.mean_squared_error(test_y, temp))
    perf_mat.loc['MAE', col] = metrics.mean_absolute_error(test_y, temp)
    perf_mat.loc['MAPE', col] = metrics.mean_absolute_percentage_error(test_y, temp)

In [47]:
perf_mat

,ALL,FORWARD,BACKWARD,STEPWISE
MSE,1557805.45026,1564189.292013,1564189.292012,1564189.292013
RMSE,1248.120767,1250.675534,1250.675534,1250.675534
MAE,866.656043,872.805236,872.805236,872.805236
MAPE,0.086998,0.087763,0.087763,0.087763


모든 변수를 넣었을 때와 3가지 방법으로 변수 선택을 했을 때 에러가 비슷하다. 전체 37개 변수를 사용했을 때와 23, 24개 정도의 변수를 사용했을 때 에러가 비슷하므로 효율성과 다중공선성 과적합 문제를 고려했을 때 후자가 신뢰성이 더 높다.